In [1]:
import shioaji as sj
from pymongo import MongoClient
from datetime import datetime, timedelta
from pandas import DataFrame
import numpy as np
from Messenger.LineMessenger import LineMessenger as Line

import warnings
warnings.filterwarnings("ignore")

import json
import os 
parent = os.path.dirname(os.path.abspath("__file__"))
StrongPath = os.path.join(parent, 'StrongTickers')
if not os.path.isdir(StrongPath):
    os.makedirs(StrongPath)

setting = {
    'user':'kevin83321',
    'pwd':'j7629864',
    'ip':'192.168.2.173',
    'port':'49153'
}

In [2]:
class DataObject:
    
    name:str=""
    pre_time:str="09:00:00"
    pre_volume:int = 0
    pre_close:int = 0
    total_v:int = 0
    q20_data:dict = {}
    q80_data:dict = {} 
    open_threshold:float = 0
    _api = None
    symbol:str = ""
    refPrice:float = 0
    entry_percent:float = .06
    exit_percent:float = .09
    sl_ratio:float = .03
    v_threshold:float = .01
    entry_threshold:float = 0
    exit_threshold:float = 0
    
    def __init__(self, contract, open_threshold:float, api=None,
                 v_threshold:float=.01, entry_percent:float=.06, exit_percent:float=.09, sl_ratio:float=.03):
        self.open_threshold = open_threshold
        self.entry_percent = entry_percent
        self.exit_percent = exit_percent
        self.sl_ratio = sl_ratio
        self.v_threshold = v_threshold
        self._initialByContract(contract)
        self._api = api
        
    def _initialByContract(self, contract):
        self.symbol = contract.code
        self.refPrice = contract.reference
        self.entry_threshold = self.refPrice * (1 + self.entry_percent)
        self.exit_threshold = self.refPrice * (1 + self.exit_percent)
        self.name = contract.name
        
    def updateQ20Dict(self, data:dict={}):
#         print("updateQ20")
        if data is None:return
        if data['simulate'] :return
        self.q20_data = data
        self.q20_data['timeStr'] = self.q20_data['datetime'].strftime("%H:%M:%S")
        self.updateSignal()
    
    def updateQ80Dict(self, data:dict):
#         print("updateQ80")
        self.q80_data = data
    
    def updatePos(self):
        pass
    
    def updateStatus(self):
        pass
    
    def updateSignal(self):
#         print("updateSignal")
        if not self.q20_data : return
        totalV = self.q20_data['totalQty']
        close = self.q20_data['close']
        volume = self.q20_data['qty']
        if self.q20_data['timeStr'] != self.pre_time:
            self.total_v = volume
            self.pre_time = self.q20_data['timeStr']
            self.pre_volume = totalV
        else:
            self.total_v += volume
        v_ratio = self.total_v / self.pre_volume
        self.pre_volume = totalV
        if not self.pre_close: 
            self.pre_close = close
            return
        
        if close >= self.open_threshold and v_ratio >= self.v_threshold and close <= self.entry_threshold and self.pre_close < self.open_threshold:
            self.sendNotify2(self.q20_data['datetime'].strftime("%H:%M:%S.%f"), self.symbol, self.name,
                             close, self.q20_data['pct_chg'])
        self.pre_close = close
        
        
    def DoTrade(self):
        if not self._api:return
    
    def sendNotify2(self, dateStr, idx, name, 
                   close, Ret):#, TVRatio, EVRatio):
        try:
            text = f'時間 : {dateStr}\n'
            text += f'股票代號/名稱 : {idx}/{name}\n'
            text += f'觸發條件 : 秒內量增價漲\n'
            text += f'成交價 : {close}\n'
            
            # 漲跌幅量
            text += f'漲跌幅 : {Ret} %\n'
#             text += f'量比(總/估) {TVRatio}/{EVRatio}\n'
            # text += '其他提醒 : \n'
            
            Line.sendMessage(text)
#             self.last_pub_time[idx] += timedelta(seconds=self.adj_time)
        except Exception as e:
            print(e)
#             self.sendError('sendNotify2')

#     def sendNotifyExit(self, dateStr, idx, name, 
#                    close, Ret, TVRatio, EVRatio, takeprofit=False):
#         try:
#             text = f'時間 : {dateStr}\n'
#             text += f'股票代號/名稱 : {idx}/{name}\n'
#             text += f'觸發出場條件 : 若有持倉，已經達到({"停利" if takeprofit else "停損"})條件囉\n'
#             text += f'成交價 : {close}\n'
            
#             # 漲跌幅量
#             text += f'漲跌幅 : {Ret} %\n'
#             text += f'量比(總/估) {TVRatio}/{EVRatio}\n'
#             # text += '其他提醒 : \n'
            
#             Line.sendMessage(text)
#             self.last_pub_time[idx] += timedelta(seconds=self.adj_time)
#         except:
#             self.sendError('sendNotify')

In [3]:
def getFollowedAssets(followedAssets=None):
    try:
        if followedAssets is None:
            table = getTable(setting, schema,'StockList')
            updateDate = sorted(table.distinct('UpdateDate'))[-1]
            datas = list(table.find({'Industry':{'$ne':''}}))
            return [x['Ticker'] for x in datas if x['UpdateDate'] == updateDate]
    except Exception as e:
        print(e)
        pass
    else:
        return followedAssets

def getTable(setting, schema, table_name):
    try:
        user = setting['user']
        pwd = setting['pwd']
        ip = setting['ip']
        port = setting['port']
        client = MongoClient(f'mongodb://{user}:{pwd}@{ip}:{port}')
        schema = client['admin'][schema]
        table = schema[table_name]
    except:
        self.sendError('getTable')
    return table

schema = 'TWSE'
# if table_name is None:
table_name = 'historicalPrice'
table = getTable(setting, schema, table_name)

In [4]:
def readStrongTicker(dtStr):
#     print(os.path.join(StrongPath, f'{dtStr}_strongTicker.json'))
#     print(os.path.isfile(os.path.join(StrongPath, f'{dtStr}_strongTicker.json')))
    if os.path.isfile(os.path.join(StrongPath, f'{dtStr}_strongTicker.json')):
        with open(os.path.join(StrongPath, f'{dtStr}_strongTicker.json'), 'r') as f:
            tickers = json.load(f)
            return tickers
    else:
        return {}

def writeStrongTicker(dtStr, tickers):
    with open(os.path.join(StrongPath, f'{dtStr}_strongTicker.json'), 'w') as f:
        json.dump(tickers,f)
        
def getStrongTickers(tradeDate = datetime.today(), entry_threshold=.06, v_threshold = 1000):

    open_thresholds = readStrongTicker(tradeDate.strftime("%Y-%m-%d"))
#     print(open_thresholds)
#     return
    if not open_thresholds:
#         _followedAssets = []
#         open_thresholds = {}
        # tickers = self.getFollowedAssets(followedAssets)

        # end_date = datetime(2021,4,27) #.today()
        tickers = getFollowedAssets()
        end_date = tradeDate - timedelta(1)
        start_date = end_date + timedelta(-60)
        table = getTable(setting, schema, table_name)
        datas = list(table.find({'Ticker':{'$in':tickers}, 'Date':{'$gte':start_date.strftime("%Y-%m-%d"), '$lte':end_date.strftime("%Y-%m-%d")}}))
        # datas = readStockDataInter(tickers,start_date, end_date)
        Master_df = DataFrame(datas)
        del Master_df['_id']

        df_map = {}
        for ticker in tickers:
            df_map[ticker] = Master_df[Master_df.Ticker==ticker]

        possible_strong = {}

        for k, df in df_map.items():
            if df.empty: continue
            if df.shape[0] < 20: continue
            # print(df)
            df['Close'] = df.Close.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['High'] = df.High.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['Open'] = df.Open.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['Low'] = df.Low.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['Volume'] = df.Volume.apply(lambda x: 0 if '-' in str(x) else x).fillna(method='ffill').astype(float)
            df['MA5'] = df.Close.rolling(5).mean()
            df['MA10'] = df.Close.rolling(10).mean()
            df['MA20'] = df.Close.rolling(20).mean()
            df['DI'] = (df.High+df.Low+2*df.Close) / 4
            df['Amp'] = (df.High-df.Low) / df.Low
            df['AmpStd'] = df.Amp.rolling(5).std()
            df['DiStd'] = df['DI'].rolling(10).std()
            df['VMA5'] = df.Volume.rolling(5).mean()
            # df['MA5Slope'] = df['MA5'].pct_change()
            # df['MA10Slope'] = df['MA10'].pct_change()
            # df['MA20Slope'] = df['MA20'].pct_change()
            MAStd = np.std([df.MA5.iloc[-1]-df.MA10.iloc[-1],df.MA5.iloc[-1]-df.MA20.iloc[-1], df.MA10.iloc[-1]-df.MA20.iloc[-1]])
            open_threshold = max(df['Close'].iloc[-5], df['Close'].iloc[-10], df['Close'].iloc[-20])
            if (open_threshold / df['Close'].iloc[-1]) < (1 + entry_threshold) and (df.VMA5.iloc[-1]) / 1000 >= v_threshold:
        #             if df['MA5Slope'].iloc[-1] >= 0.01:# and df['MA10Slope'].iloc[-1] >= 0 and df['MA20Slope'].iloc[-1] >= 0:
                temp_ma5 = np.mean(df['Close'].iloc[-4:].tolist() + [open_threshold])
                possible_strong[k] = (open_threshold, MAStd, df.DiStd.iloc[-1]/df['Close'].iloc[-1], df.AmpStd.iloc[-1])
#                 _followedAssets.append(k)
                open_thresholds[k] = open_threshold
        writeStrongTicker(tradeDate.strftime("%Y-%m-%d"), open_thresholds)
    return open_thresholds

In [5]:
open_thresholds = getStrongTickers()
len(open_thresholds.keys())

494

In [6]:
api1 = sj.Shioaji()

In [7]:
api1.login("F128497445", "j7629864")

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


[FutureAccount(person_id='F128497445', broker_id='F002000', account_id='1473661', signed=True, username='鄭圳宏'),
 Account(account_type=<AccountType.H: 'H'>, person_id='F128497445', broker_id='9A92', account_id='0011645', username='鄭圳宏'),
 StockAccount(person_id='F128497445', broker_id='9A92', account_id='0231901', signed=True, username='鄭圳宏\u3000\u3000')]

In [8]:
from shioaji import TickSTKv1, Exchange, BidAskSTKv1, TickFOPv1, BidAskFOPv1

@api1.on_tick_stk_v1()
@api1.on_bidask_stk_v1()
@api1.on_bidask_fop_v1()
@api1.on_tick_fop_v1()
def quote_callback(exchange:Exchange, tick:[TickSTKv1, BidAskSTKv1, TickFOPv1, BidAskFOPv1]):
    try:
        if type(tick) in [TickSTKv1, TickFOPv1]:
            NotifyTickers[ticker] = NotifyTickers.get(ticker, DataObject(tmp_contract, open_threshold=17655, api=api1))
            NotifyTickers[ticker].updateQ20Dict(dict(
                symbol=tick.code,
                datetime=tick.datetime,
                open=float(tick.open),
                high=float(tick.high),
                low=float(tick.low),
                close=float(tick.close),
                avg_price=float(tick.avg_price),
                qty=int(tick.volume),
                totalQty=int(tick.total_volume),
                pct_chg=float(tick.pct_chg),
                simulate=bool(tick.simtrade),
            ))
#             print(f"Exchange : {exchange}, Tick:{NotifyTickers[ticker].q20_data}\n")
        if type(tick) in [BidAskSTKv1, BidAskFOPv1]:
            NotifyTickers[ticker].updateQ80Dict(dict(
                symbol = tick.code,
                datetime = tick.datetime,
                bid1 = float(tick.bid_price[0]),
                bid2 = float(tick.bid_price[1]),
                bid3 = float(tick.bid_price[2]),
                bid4 = float(tick.bid_price[3]),
                bid5 = float(tick.bid_price[4]),
                bidQty1 = float(tick.bid_volume[0]),
                bidQty2 = float(tick.bid_volume[1]),
                bidQty3 = float(tick.bid_volume[2]),
                bidQty4 = float(tick.bid_volume[3]),
                bidQty5 = float(tick.bid_volume[4]),
                askQty1 = float(tick.ask_volume[0]),
                askQty2 = float(tick.ask_volume[1]),
                askQty3 = float(tick.ask_volume[2]),
                askQty4 = float(tick.ask_volume[3]),
                askQty5 = float(tick.ask_volume[4]),
                ask1 = float(tick.ask_price[0]),
                ask2 = float(tick.ask_price[1]),
                ask3 = float(tick.ask_price[2]),
                ask4 = float(tick.ask_price[3]),
                ask5 = float(tick.ask_price[4]),
            ))
#             print(f"Exchange : {exchange}, BidAsk : {NotifyTickers[ticker].q80_data}\n")
    except Exception as e:
        print(e)


# def fop_quote_callback(topic, msg):
#     print(f"Exchange : {topic}, Data : {msg}")
    
api1.quote.set_on_tick_stk_v1_callback(quote_callback)
api1.quote.set_on_bidask_stk_v1_callback(quote_callback)
api1.quote.set_on_bidask_fop_v1_callback(quote_callback)
api1.quote.set_on_tick_fop_v1_callback(quote_callback)

In [9]:
NotifyTickers = {}
for i in range(125):
    ticker = sorted(open_thresholds.keys())[i]
    # ticker = "TXFL1"
    tmp_contract = api1.Contracts.Stocks[ticker]
    if tmp_contract.day_trade  == sj.constant.DayTrade.No:continue
    # tmp_contract = api1.Contracts.Futures[ticker]
    NotifyTickers[ticker] = NotifyTickers.get(ticker, DataObject(tmp_contract, open_threshold=17655, api=api1))
    api1.quote.subscribe(
        tmp_contract, 
        quote_type = sj.constant.QuoteType.Tick, # or 'tick'
        version = sj.constant.QuoteVersion.v1 # or 'v1'
    )
    api1.quote.subscribe(
        tmp_contract, 
        quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
        version = sj.constant.QuoteVersion.v1 # or 'v1'
    )

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1101 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1101 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1210 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1210 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1216 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1216 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1217 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1217 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1229 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1229 | Event

In [10]:
from time import sleep
while 1:
    try:
        sleep(1)
    except:
        break


Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1326 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1402 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1402 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1409 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1409 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1414 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1414 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1434 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/1434 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/1440 | Even

Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2017 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2020 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2020 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2022 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2022 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2023 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2023 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2027 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2027 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2028 | Event

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2368 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2368 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2369 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2369 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2371 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2371 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2374 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2374 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/2375 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/2375 | Event